In [1]:
import os
import json
import torch
import numpy as np
import xml.etree.ElementTree as ET
from pathlib import Path
from PIL import Image

# 評估函數
def calculate_metrics(detections, annotations, iou_threshold=0.5):
    TP = 0
    FP = 0
    FN = 0
    iou_list = []

    for detection in detections:
        detected_bbox = detection['bbox']
        detected_filename = detection['file_name']
        matched = False
        for annotation in annotations:
            if annotation['file_name'] == detected_filename:
                true_bbox = annotation['bbox']
                iou = compute_iou(detected_bbox, true_bbox)
                if iou >= iou_threshold:
                    TP += 1
                    matched = True
                    iou_list.append(iou)
                    break
        if not matched:
            FP += 1
    
    FN = len(annotations) - TP

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    map_score = np.mean(iou_list) if iou_list else 0

    return precision, recall, f1_score, map_score

def compute_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)

    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    iou = interArea / float(boxAArea + boxBArea - interArea)

    return iou

# 文件路徑配置
xml_folder = 'C:/Users/ediso/Desktop/ML_project3/container/Test_xml'
image_folder = 'C:/Users/ediso/Desktop/ML_project3/container/Test'
output_json = "C:/Users/ediso/Desktop/ML_project3/annotations_custom.json"
model_path = 'C:/Users/ediso/Desktop/ML_project3/best.pt'
test_image_folder = 'C:/Users/ediso/Desktop/ML_project3/container/Test'
detections_json = "C:/Users/ediso/Desktop/ML_project3/detections_custom.json"

# 加載已訓練模型
model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# 加載測試圖像
test_images = [f for f in os.listdir(test_image_folder) if f.endswith(".jpg") or f.endswith(".png")]

all_detections = []

for image_file in test_images:
    image_path = os.path.join(test_image_folder, image_file)
    results = model(image_path)
    predictions = results.xyxy[0].cpu().numpy()  # xyxy 格式

    for det in predictions:
        if det[4] > 0.5:  # 置信度閾值
            x1, y1, x2, y2, score, class_id = det
            all_detections.append({
                "file_name": image_file,
                "category": model.names[int(class_id)],
                "bbox": [float(x1), float(y1), float(x2), float(y2)],
                "score": float(score)
            })

print("檢測結果:", all_detections)

with open(detections_json, 'w') as f:
    json.dump(all_detections, f, indent=4)

with open(output_json, 'r') as f:
    annotations = json.load(f)

precision, recall, f1_score, map_score = calculate_metrics(all_detections, annotations)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1_score:.4f}")
print(f"mAP: {map_score:.4f}")


Using cache found in C:\Users\ediso/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-5-24 Python-3.12.1 torch-2.2.2+cu118 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


檢測結果: [{'file_name': 'image_0001.jpg', 'category': 'container', 'bbox': [744.66162109375, 113.20668029785156, 1113.485107421875, 233.2898712158203], 'score': 0.9139329195022583}, {'file_name': 'image_0002.jpg', 'category': 'container', 'bbox': [748.35498046875, 157.676513671875, 1155.4140625, 294.2265930175781], 'score': 0.9062440991401672}, {'file_name': 'image_0003.jpg', 'category': 'container', 'bbox': [746.9759521484375, 156.09298706054688, 1152.880859375, 288.89251708984375], 'score': 0.918773889541626}, {'file_name': 'image_0004.jpg', 'category': 'container', 'bbox': [785.4134521484375, 96.75732421875, 1125.44921875, 209.44606018066406], 'score': 0.9179355502128601}, {'file_name': 'image_0005.jpg', 'category': 'container', 'bbox': [826.54296875, 43.44511413574219, 1096.3280029296875, 135.93209838867188], 'score': 0.9516063332557678}, {'file_name': 'image_0006.jpg', 'category': 'container', 'bbox': [848.5709228515625, 7.191925048828125, 1068.1748046875, 84.27560424804688], 'score'